In [1]:
import json
from tqdm import tqdm
import pandas as pd
from utils.preprocessing import *

In [2]:
start_index = 0
end_index = 25000

In [3]:
# get running data from endomondo
input_file = "archive/endomondoHR_proper.json"
output_file = "archive/endomondoHR_fixed.json"

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        corrected_line = line.replace("'", '"')  # Replace single quotes with double quotes
        outfile.write(corrected_line)


data = []
with open(output_file, "r", encoding="utf-8") as f:
    for line in tqdm(f):
        data.append(json.loads(line))  # Load each line separately

print(f"Loaded {len(data)} JSON objects")


167783it [00:42, 3980.37it/s]

Loaded 167783 JSON objects


In [4]:
# limit to only running data and specified index range
tqdm.pandas()
df = pd.DataFrame(data)
run_routes = df[df['sport']=='run']

run_routes.reset_index(inplace=True, drop=True)

run_routes['distance'] = run_routes.progress_apply(lambda row: get_route_distance(row['latitude'], row['longitude']), axis=1)
run_routes['duration'] = run_routes.apply(lambda row: calculate_route_duration(row['timestamp']), axis=1)
run_routes['start_end_dist'] = run_routes.apply(lambda row: get_start_end_dist(row['latitude'], row['longitude']), axis=1)

# ensure indexes are in bounds
start_index = max(0, start_index)
end_index = min(end_index, len(run_routes))

run_routes = run_routes.iloc[start_index:end_index, :]
run_routes.head()

100%|██████████| 70591/70591 [00:14<00:00, 4788.73it/s]
/var/folders/d4/c154mt9s11x5q965j91l06qw0000gn/T/ipykernel_1154/4208802304.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_routes['distance'] = run_routes.progress_apply(lambda row: get_route_distance(row['latitude'], row['longitude']), axis=1)
/var/folders/d4/c154mt9s11x5q965j91l06qw0000gn/T/ipykernel_1154/4208802304.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_routes['duration'] = run_routes.apply(lambda row: calculate_route_durati

,longitude,altitude,latitude,sport,id,heart_rate,gender,timestamp,url,userId,speed,distance,duration,start_end_dist
0,"[6.8854929, 6.8853678, 6.8851621, 6.8848205, 6...","[-173.8, -151.2, -161.6, -165.4, -168.6, -172....","[52.2226809, 52.222727, 52.2228258, 52.2228606...",run,321063199,"[80, 81, 94, 100, 102, 112, 108, 114, 110, 109...",male,"[1397079203, 1397079210, 1397079218, 139707922...",https://www.endomondo.com/users/4969375/workou...,4969375,NaN,5.361321,64.800000,0.094728
1,"[6.9144073, 6.9142929, 6.9141539, 6.9140268, 6...","[57.8, 57.6, 57.0, 56.4, 55.8, 55.2, 54.4, 53....","[52.2111711, 52.2112631, 52.2114064, 52.211608...",run,303565793,"[60, 62, 92, 92, 132, 150, 150, 159, 159, 161,...",male,"[1393908533, 1393908541, 1393908549, 139390855...",https://www.endomondo.com/users/4969375/workou...,4969375,NaN,8.844797,70.066667,0.002572
2,"[6.9141348, 6.9145702, 6.9151684, 6.9158377, 6...","[22.8, 26.4, 30.8, 35.6, 43.0, 48.4, 49.8, 49....","[52.2110297, 52.2106325, 52.2102453, 52.209833...",run,302666522,"[77, 93, 107, 121, 118, 120, 120, 124, 124, 12...",male,"[1393687929, 1393687948, 1393687967, 139368798...",https://www.endomondo.com/users/4969375/workou...,4969375,NaN,18.246353,168.766667,0.014032
3,"[6.8678543, 6.8678634, 6.8675429, 6.8672183, 6...","[35.4, 35.2, 34.6, 34.2, 35.0, 35.2, 34.8, 34....","[52.1936673, 52.1934354, 52.1931993, 52.192873...",run,296982347,"[75, 101, 116, 120, 124, 126, 127, 129, 126, 1...",male,"[1392480163, 1392480176, 1392480189, 139248020...",https://www.endomondo.com/users/4969375/workou...,4969375,NaN,13.584195,108.383333,0.004041
4,"[6.9143328, 6.9146396, 6.9148949, 6.9151568, 6...","[63.0, 65.2, 66.0, 66.2, 65.8, 65.8, 67.0, 67....","[52.2112195, 52.2110264, 52.2108135, 52.210601...",run,295890426,"[58, 83, 112, 115, 117, 116, 141, 121, 120, 11...",male,"[1392180426, 1392180436, 1392180446, 139218045...",https://www.endomondo.com/users/4969375/workou...,4969375,NaN,10.438034,85.300000,0.010509


In [5]:
# run_routes.to_csv('run_routes_unprocessed.csv')

In [6]:
# import json
# from tqdm import tqdm
# import pandas as pd
# from utils.preprocessing import *

# run_routes = pd.read_csv('run_routes_unprocessed.csv')


In [7]:
# run_routes['longitude'] = run_routes['longitude'].apply(lambda x: json.loads(x))
# run_routes['latitude'] = run_routes['latitude'].apply(lambda x: json.loads(x))
# run_routes.set_index('Unnamed: 0', inplace=True)

In [ ]:

processed_routes = parallel_process(run_routes)


  0%|          | 36/25000 [01:20<12:57:38,  1.87s/it]

  0%|          | 61/25000 [02:06<12:29:46,  1.80s/it]

  0%|          | 84/25000 [02:49<12:51:33,  1.86s/it]

  0%|          | 92/25000 [03:04<12:54:34,  1.87s/it]

  0%|          | 94/25000 [03:07<12:36:35,  1.82s/it]

...

 28%|██▊       | 7020/25000 [3:48:46<10:17:20,  2.06s/it]

 28%|██▊       | 7049/25000 [3:49:46<10:09:53,  2.04s/it]

 28%|██▊       | 7072/25000 [3:50:35<11:02:44,  2.22s/it]

 28%|██▊       | 7086/25000 [3:51:05<10:36:21,  2.13s/it]

 62%|██████▏   | 15441/25000 [7:32:53<4:21:31,  1.64s/it]

In [1]:

folder_name = "data"

# Check if the folder already exists
if not os.path.exists(folder_name):
    # Create the folder
    os.makedirs(folder_name)

processed_routes.to_csv(f'data/processed_routes_{start_index}_{end_index}.csv', index=False)
print(f"Processed routes saved to data/processed_routes_{start_index}_{end_index}.csv")

NameError: name 'os' is not defined